In [20]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import re
import numpy as np

import pickle

In [2]:
# Read the data
data = pd.read_csv('/content/Sentiment.csv')

# Maintaining required columns only
data = data[['text','sentiment']]

# Excluding Neutral sentiment
data = data[data.sentiment != 'Neutral']
data.head()

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative


In [3]:
data.sentiment.value_counts()

Negative    8493
Positive    2236
Name: sentiment, dtype: int64

In [4]:
data.isnull().sum()

text         0
sentiment    0
dtype: int64

In [5]:
# Lowering all the characters in the data
data['text'] = data['text'].apply(lambda x: x.lower())

# Removing special characyers from data
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [6]:
# Removing retweet
for idx, row in data.iterrows():
  row[0] = row[0].replace('rt','')

# Tokenization
max_features = 2000

# Splitting text to words
tokenizer = Tokenizer(num_words = max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)

# Converting texts to sentences
X = tokenizer.texts_to_sequences(data['text'].values)

# Padding
X = pad_sequences(X)

In [9]:
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer ='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 28, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________


In [10]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
Y = to_categorical(integer_encoded)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42) 
print(X_train.shape,Y_train.shape) 
print(X_test.shape,Y_test.shape) 

batch_size = 32 
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) 

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)
225/225 - 39s - loss: 0.4380 - accuracy: 0.8187 - 39s/epoch - 175ms/step


In [11]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size) 
print("score: %.2f" % (score)) 
print("acc: %.2f" % (acc)) 

111/111 - 3s - loss: 0.3690 - accuracy: 0.8402 - 3s/epoch - 31ms/step
score: 0.37
acc: 0.84


In [12]:
model.save('tweet_model.h5') 

In [21]:
# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [38]:
from keras.models import load_model
seq_model = load_model('tweet_model.h5')



tweet = ['A lot of good things are happening. We are respected again throughout the world, and thats a great thing.@realDonaldTrump is a good person with lot of love and']
print(tweet)




#enhancing the text data
for i in range(len(tweet)):
    tweet[i] = tweet[i].lower() #changes all the upper characters to lower characters
    tweet[i] = re.sub('[^a-zA-z0-9\s]','', tweet[i]) #removes the special characters 


#Tokenization
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ') #splitting the text to tokens
tokenizer.fit_on_texts(tweet)

#print(tweet)
tweet = tokenizer.texts_to_sequences(tweet) #converting texts to sequences 
tweet = pad_sequences(tweet)



prediction = model.predict([tweet])
print(prediction)

def decode_prediction(prediction):
    return 'Negative' if prediction.all() < 0.5 else 'Positive'

decode_prediction(prediction)



['A lot of good things are happening. We are respected again throughout the world, and thats a great thing.@realDonaldTrump is a good person with lot of love and']
[[0.9445524  0.05544754]]


'Positive'